# ConvNet model

Building a simple ConvNet model for multi-label classification using Keras(Luda)

Adapted from "Simple Keras Starter" Kernel from anokas on Kaggle


In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.metrics import fbeta_score, precision_score, make_scorer, average_precision_score
import matplotlib.pyplot as plt
import cv2
import warnings

%pylab inline
pylab.rcParams['figure.figsize'] = (15, 10)


import cv2
from tqdm import tqdm

Populating the interactive namespace from numpy and matplotlib


In [84]:
# General functions from 0-baseline notebook

# Preprocess images
# Input: number of samples, dim, path
# Output: X, y, list of class names
def preprocess(n_samples, rescaled_dim, f_path ="../data/train.csv"):
    df = pd.read_csv(f_path)
    df['split_tags'] = df['tags'].map(lambda row: row.split(" "))
    lb = MultiLabelBinarizer()
    y = lb.fit_transform(df['split_tags'])
    y = y[:n_samples]
    
    imgs = []
    # for each image, rescale to same dimension and flattern
    print "processing images..."
    count = 0
    for name in df.head(n_samples)['image_name'].values:
        raw_img = plt.imread('../data/train-jpg/{}.jpg'.format(name))
        #print raw_img
        imgs.append(cv2.resize(raw_img, (rescaled_dim, rescaled_dim), cv2.INTER_LINEAR).reshape(1, -1))
        count+=1
        if count % 1000 == 0:
            print count, "processed"

    # remove dimenions    
    X = np.squeeze(np.array(imgs))

    # scale X so that each feature be between 0 and 1
    X = MinMaxScaler().fit_transform(np.float32(X))
    
    return np.array(X, np.float16), np.array(y, np.uint8), lb.classes_


# Using a trained model, calculate the F2_score
# input: X_test, y_test, model
# output: a list of F2_score by class(length 17)
def calc_F2_score_cnn(X_test, y_test, model, thres = 0.2):
    predicted = np.array(model.predict(X_test)) > thres
    score = fbeta_score(y_test, predicted, beta=2, average=None)
    return score

def print_results(scores, ind_to_classes):
    # print the scores with the classes
    results = [(ind_to_classes[l], scores[l]) for l in scores.argsort()[::-1]]
    print "************************RESULTS************************"
    for res in results:
        print res[0], res[1]
    print "************************END RESULTS************************"
    
# define cross validation
# if fold = 1 just use a 20% test set
# Training model func: need to take in X_train, y_train, and initial model(if necessary)
def cross_validation_train_score(X, y, train_model_func, scoring_func, folds=10):

    if folds == 1:
        print "Using a 20% test set..."
        print "X shape:", X.shape
        print "y shape:", y.shape
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
        model = train_model_func(X_train, y_train)
        score = scoring_func(X_test, y_test, model)
        return model, np.array(score)

    print "Using cross validation with folds", folds
    print "X shape:", X.shape
    print "y shape:", y.shape    
    kf = KFold(n_splits=folds, shuffle=True)
    scores_all = []
    for ind, (train, test) in enumerate(kf.split(X)):
        print "Fold ", ind
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]

        model = train_model_func(X_train, y_train)
        score = scoring_func(X_test, y_test, model)
        scores_all.append(score)
        
    #TODO: change to return highest accurac model
    return model, np.mean(np.array(scores_all), axis=0)

In [65]:
# CNN specific preprocessing function, creating 4 dimensional input vectors
def preprocess_cnn(n_samples, rescaled_dim, f_path ="../data/train.csv"):
    df = pd.read_csv(f_path)
    df['split_tags'] = df['tags'].map(lambda row: row.split(" "))
    lb = MultiLabelBinarizer()
    y = lb.fit_transform(df['split_tags'])
    y = y[:n_samples]
    
    imgs = []
    # for each image, rescale to same dimension and flattern
    print "processing images..."
    count = 0
    for name in df.head(n_samples)['image_name'].values:
        raw_img = plt.imread('../data/train-jpg/{}.jpg'.format(name))
        #print raw_img
        imgs.append(cv2.resize(raw_img, (rescaled_dim, rescaled_dim), cv2.INTER_LINEAR)[:, :, :3])
        count+=1
        if count % 1000 == 0:
            print count, "processed"

    # remove dimenions, normalize  
    X = np.squeeze(np.array(imgs)) / 255.

    return np.array(X, np.float16), np.array(y, np.uint8), lb.classes_



# Define model architecture

In [87]:
def train_cnn_model_1(x_train, y_train):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(32, 32, 3)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(17, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
                  optimizer='adam',
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=64,
              epochs=5,
              verbose=1)
    return model

# Train the model!

In [107]:
NUM_TRAIN = 40482
NUM_SAMPLE = NUM_TRAIN
DIM = 32
X, y, ind_to_classes = preprocess_cnn(NUM_SAMPLE, DIM, "../data/train.csv")
print X.shape, y.shape

processing images...
1000 processed
2000 processed
3000 processed
4000 processed
5000 processed
6000 processed
7000 processed
8000 processed
9000 processed
10000 processed
11000 processed
12000 processed
13000 processed
14000 processed
15000 processed
16000 processed
17000 processed
18000 processed
19000 processed
20000 processed
21000 processed
22000 processed
23000 processed
24000 processed
25000 processed
26000 processed
27000 processed
28000 processed
29000 processed
30000 processed
31000 processed
32000 processed
33000 processed
34000 processed
35000 processed
36000 processed
37000 processed
38000 processed
39000 processed
40000 processed
(40479, 32, 32, 3) (40479, 17)


In [108]:
model, cv_f2_scores = cross_validation_train_score(X, y, train_cnn_model_1, calc_F2_score_cnn, 1) #no cross-validation
print_results(cv_f2_scores, ind_to_classes)

Using a 20% test set...
X shape: (40479, 32, 32, 3)
y shape: (40479, 17)
Epoch 1/5
32383/32383 [==============================] - 78s - loss: 0.2342 - acc: 0.9103    
Epoch 2/5
32383/32383 [==============================] - 78s - loss: 0.1947 - acc: 0.9237    
Epoch 3/5
32383/32383 [==============================] - 78s - loss: 0.1844 - acc: 0.9280    
Epoch 4/5
32383/32383 [==============================] - 77s - loss: 0.1773 - acc: 0.9308    
Epoch 5/5
32383/32383 [==============================] - 79s - loss: 0.1708 - acc: 0.9336    
************************RESULTS************************
primary 0.987326887902
clear 0.95884620587
agriculture 0.812865085509
partly_cloudy 0.787007318013
road 0.73715675347
cloudy 0.670297444491
haze 0.609621740727
water 0.604987684729
habitation 0.571746384872
cultivation 0.404491931967
artisinal_mine 0.387409200969
bare_ground 0.0298329355609
slash_burn 0.0
conventional_mine 0.0
selective_logging 0.0
blow_down 0.0
blooming 0.0
***********************

In [109]:
#save model
import datetime
def timestamp():
    return '{:%m%d_%H_%M_%s}'.format(datetime.datetime.now())

fn = "../data/cnn-1_" + timestamp() + '.h5'
model.save(fn)